In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
# Load the input data
mol_train = pd.read_csv('data/mol_train_uff.csv.gz')
mol_valid = pd.read_csv('data/mol_valid_uff.csv.gz')
mol_test = pd.read_csv('data/mol_test_uff.csv.gz')

smiles_train = pd.read_csv('data/smiles_train.csv.gz', index_col=0)
smiles_valid = pd.read_csv('data/smiles_valid.csv.gz', index_col=0)
smiles_test = pd.read_csv('data/smiles_test.csv.gz', index_col=0)

In [3]:
from keras.models import load_model
from nfp import custom_layers
from nfp.preprocessing import GraphSequence
import warnings
import pickle
from tqdm import tqdm

Using TensorFlow backend.


In [4]:
props = ['gap', 'homo', 'lumo', 'spectral_overlap', 'homo_extrapolated',
         'lumo_extrapolated', 'gap_extrapolated', 'optical_lumo_extrapolated']

def mae(true, pred):
    return np.nanmean(np.abs(true - pred), 0)

In [27]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = load_model('b3lyp_2D_mae_multitarget/best_model.hdf5',
                       custom_objects=custom_layers)
    
with open('b3lyp_2D_mae_multitarget/preprocessor.p', 'rb') as f:
    preprocessor = pickle.load(f)
    
with open('b3lyp_2D_mae_multitarget/y_scaler.p', 'rb') as f:
    y_scaler = pickle.load(f)

inputs_test = preprocessor.predict(smiles_test.smile)
test_inputs_seq = GraphSequence(inputs_test, shuffle=False, final_batch=True, batch_size=128)


100%|██████████| 5000/5000 [00:08<00:00, 585.17it/s]

In [28]:
inputs_train = preprocessor.predict(smiles_train.smile)


  9%|▉         | 7425/80823 [00:14<01:46, 690.04it/s]


 19%|█▉        | 15549/80823 [00:29<02:33, 425.24it/s]


 27%|██▋       | 22001/80823 [00:44<02:04, 472.93it/s]


 35%|███▌      | 28375/80823 [00:59<02:02, 426.68it/s]


 43%|████▎     | 34646/80823 [01:13<01:50, 416.79it/s]


...

 73%|███████▎  | 59206/80823 [02:12<00:51, 417.31it/s]


 81%|████████  | 65361/80823 [02:27<00:39, 391.46it/s]


 88%|████████▊ | 71368/80823 [02:42<00:22, 429.67it/s]


 96%|█████████▌| 77335/80823 [02:56<00:08, 398.01it/s]


100%|██████████| 80823/80823 [03:05<00:00, 435.73it/s]

In [29]:
max((item['n_bond'] for item in inputs_train))

424

In [30]:
y_pred_test = y_scaler.inverse_transform(
    model.predict_generator(test_inputs_seq, verbose=True))

y_true = smiles_test.set_index('smile').values
mae_2d = pd.Series({props[i]: mae(y_pred_test[:, i], y_true[:, i]) for i in range(8)},
                     name='b3lyp_2D_mae_multitarget')

mae_2d = mae_2d * 1000
mae_2d['spectral_overlap'] /= 1000
mae_2d.round(1)

40/40 [==============================] - 13s 319ms/step


gap                           35.4
homo                          29.4
lumo                          29.2
spectral_overlap             149.2
homo_extrapolated             47.4
lumo_extrapolated             46.8
gap_extrapolated              56.3
optical_lumo_extrapolated     43.9
Name: b3lyp_2D_mae_multitarget, dtype: float64

In [10]:
y_multitarget = pd.DataFrame(y_pred_test, columns=props)

In [11]:
st_predictions = []

for prop in props:
    print(prop)
    
    model_name = 'b3lyp_2d_{}'.format(prop)

    with open('{}/preprocessor.p'.format(model_name), 'rb') as f:
        preprocessor = pickle.load(f)
        
    with open('{}/y_scaler.p'.format(model_name), 'rb') as f:
        y_scaler = pickle.load(f)

    inputs_test = preprocessor.predict(smiles_test.smile)
    test_inputs_seq = GraphSequence(inputs_test, shuffle=False, final_batch=True, batch_size=128)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    y_pred_test = y_scaler.inverse_transform(
        model.predict_generator(test_inputs_seq, verbose=True))
    st_predictions += [y_pred_test]

  1%|          | 57/5000 [00:00<00:08, 559.86it/s]

gap


100%|██████████| 5000/5000 [00:08<00:00, 618.00it/s]


40/40 [==============================] - 1s 36ms/step


  0%|          | 0/5000 [00:00<?, ?it/s]

...

40/40 [==============================] - 2s 54ms/step


  0%|          | 0/5000 [00:00<?, ?it/s]

optical_lumo_extrapolated


100%|██████████| 5000/5000 [00:07<00:00, 671.40it/s]


40/40 [==============================] - 2s 58ms/step


In [12]:
y_2d_st = pd.DataFrame([i.flatten() for i in st_predictions], index=props).T

mae_2d_st = pd.Series({props[i]: mae(smiles_test[props].values[:, i], y_2d_st.values[:, i]) for i in range(8)},
                         name='b3lyp_2d_st')

mae_2d_st = mae_2d_st * 1000
mae_2d_st['spectral_overlap'] /= 1000
mae_2d_st.round(1)

gap                           36.9
homo                          32.1
lumo                          27.9
spectral_overlap             149.3
homo_extrapolated             49.1
lumo_extrapolated             47.8
gap_extrapolated              57.1
optical_lumo_extrapolated     47.8
Name: b3lyp_2d_st, dtype: float64

In [13]:
def rbf_expansion(distances, mu=0, delta=0.2, kmax=150):
    k = np.arange(0, kmax)
    logits = -(np.atleast_2d(distances).T - (-mu + delta * k))**2 / delta
    return np.exp(logits)

def precalc_rbfs(inputs):

    for item in tqdm(inputs):

        item['distance_rbf'] = rbf_expansion(item['distance'])
        del item['distance']

    return inputs

In [14]:
from rdkit.Chem import MolFromMolBlock
from nfp.preprocessing import RobustNanScaler

schnet_predictions = []
for prop in props:
    print(prop)
    
    model_name = 'b3lyp_schnet2_{}'.format(prop)

    with open('{}/schnet_preprocessor.p'.format(model_name), 'rb') as f:
        schnet_preprocessor = pickle.load(f)

    inputs_test = schnet_preprocessor.predict((MolFromMolBlock(mol) for _, mol in mol_test.mol.iteritems()))
    rbf_inputs_test = precalc_rbfs(list(inputs_test))
    rbf_input_seq = GraphSequence(rbf_inputs_test, shuffle=False, batch_size=32)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    itrain = mol_train[mol_train[prop].notna()]

    # Rescale Y matrix
    y_train_raw = itrain[[prop]].values

    y_scaler = RobustNanScaler()
    y_scaler.fit(y_train_raw)

    y_pred = model.predict_generator(rbf_input_seq, verbose=True)
    schnet_predictions += [y_scaler.inverse_transform(y_pred)]

31it [00:00, 299.56it/s]

gap


5000it [00:16, 310.85it/s]
100%|██████████| 5000/5000 [00:16<00:00, 297.04it/s]


157/157 [==============================] - 9s 57ms/step
homo


5000it [00:16, 307.66it/s]
100%|██████████| 5000/5000 [00:17<00:00, 288.46it/s]


...

157/157 [==============================] - 10s 66ms/step


0it [00:00, ?it/s]

optical_lumo_extrapolated


5000it [00:16, 306.87it/s]
100%|██████████| 5000/5000 [00:16<00:00, 301.61it/s]


157/157 [==============================] - 10s 64ms/step


In [15]:
y_schnet = pd.DataFrame([i.flatten() for i in schnet_predictions], index=props).T

In [16]:
mae_schnet = pd.Series({props[i]: mae(mol_test[props].values[:, i], y_schnet.values[:, i]) for i in range(8)},
                         name='b3lyp_schnet')
mae_schnet = mae_schnet * 1000
mae_schnet['spectral_overlap'] /= 1000
mae_schnet.round(1)

gap                          32.7
homo                         27.0
lumo                         24.8
spectral_overlap             96.6
homo_extrapolated            56.9
lumo_extrapolated            56.8
gap_extrapolated             69.8
optical_lumo_extrapolated    57.2
Name: b3lyp_schnet, dtype: float64

# Predictions using UFF reoptimized molecules

In [20]:
mol_test_uff = pd.read_csv('data/mol_test_uff.csv.gz')
mol_test_uff = mol_test_uff[~mol_test_uff['molUFF'].isna()]

In [21]:
schnet_predictions_uff = []
for prop in props:
    print(prop)
    
    model_name = 'b3lyp_schnet2_uff_{}'.format(prop)

    with open('{}/schnet_preprocessor.p'.format(model_name), 'rb') as f:
        schnet_preprocessor = pickle.load(f)

    inputs_test = schnet_preprocessor.predict((MolFromMolBlock(mol) for _, mol in mol_test_uff.molUFF.iteritems()))
    rbf_inputs_test = precalc_rbfs(list(inputs_test))
    rbf_input_seq = GraphSequence(rbf_inputs_test, shuffle=False, batch_size=32)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = load_model('{}/best_model.hdf5'.format(model_name),
                           custom_objects=custom_layers)

    itrain = mol_train[mol_train[prop].notna()]

    # Rescale Y matrix
    y_train_raw = itrain[[prop]].values

    y_scaler = RobustNanScaler()
    y_scaler.fit(y_train_raw)

    y_pred = model.predict_generator(rbf_input_seq, verbose=True)
    schnet_predictions_uff += [y_scaler.inverse_transform(y_pred)]


0it [00:00, ?it/s]
30it [00:00, 293.77it/s]

gap



59it [00:00, 292.34it/s]
90it [00:00, 295.20it/s]
119it [00:00, 291.51it/s]
145it [00:00, 279.34it/s]
177it [00:00, 288.98it/s]
205it [00:00, 286.10it/s]
232it [00:00, 273.22it/s]
260it [00:00, 275.15it/s]
289it [00:01, 279.08it/s]
318it [00:01, 280.98it/s]
349it [00:01, 287.52it/s]
382it [00:01, 297.85it/s]
412it [00:01, 295.58it/s]
443it [00:01, 295.37it/s]
473it [00:01, 285.31it/s]
502it [00:01, 284.90it/s]
531it [00:01, 283.25it/s]
562it [00:01, 290.16it/s]
592it [00:02, 290.27it/s]
622it [00:02, 284.70it/s]
651it [00:02, 282.44it/s]
681it [00:02, 285.71it/s]
711it [00:02, 288.50it/s]
742it [00:02, 292.42it/s]
772it [00:02, 291.10it/s]
802it [00:02, 293.64it/s]
833it [00:02, 296.26it/s]
863it [00:02, 297.06it/s]
893it [00:03, 293.82it/s]
923it [00:03, 283.23it/s]
952it [00:03, 278.50it/s]
984it [00:03, 287.21it/s]
1013it [00:03, 287.16it/s]
1044it [00:03, 288.49it/s]
1077it [00:03, 299.73it/s]
1108it [00:03, 298.36it/s]
1138it [00:03, 292.04it/s]
1169it [00:04, 294.42it/s]
1199it 

100%|██████████| 4996/4996 [00:15<00:00, 312.58it/s]

157/157 [==============================] - 11s 69ms/step


...


0it [00:00, ?it/s]

optical_lumo_extrapolated



25it [00:00, 244.80it/s]
54it [00:00, 253.45it/s]
86it [00:00, 267.96it/s]
114it [00:00, 270.71it/s]
142it [00:00, 271.04it/s]
173it [00:00, 280.44it/s]
202it [00:00, 283.18it/s]
229it [00:00, 267.79it/s]
256it [00:00, 267.23it/s]
287it [00:01, 277.44it/s]
315it [00:01, 276.27it/s]
347it [00:01, 287.21it/s]
380it [00:01, 297.41it/s]
410it [00:01, 295.63it/s]
440it [00:01, 294.71it/s]
470it [00:01, 286.70it/s]
499it [00:01, 287.07it/s]
528it [00:01, 283.96it/s]
560it [00:01, 292.07it/s]
590it [00:02, 288.52it/s]
619it [00:02, 286.75it/s]
648it [00:02, 282.01it/s]
677it [00:02, 282.50it/s]
708it [00:02, 288.43it/s]
737it [00:02, 287.91it/s]
767it [00:02, 290.71it/s]
797it [00:02, 289.12it/s]
827it [00:02, 291.42it/s]
857it [00:02, 293.37it/s]
887it [00:03, 287.71it/s]
916it [00:03, 277.03it/s]
945it [00:03, 279.90it/s]
975it [00:03, 285.16it/s]
1005it [00:03, 286.17it/s]
1034it [00:03, 284.16it/s]
1064it [00:03, 287.76it/s]
1096it [00:03, 296.07it/s]
1126it [00:03, 290.06it/s]
1157it [0

100%|██████████| 4996/4996 [00:16<00:00, 297.76it/s]

157/157 [==============================] - 15s 96ms/step


In [22]:
y_schnet_uff = pd.DataFrame([i.flatten() for i in schnet_predictions_uff], index=props).T

In [31]:
mae_schnet_uff = pd.Series({props[i]: mae(
    mol_test_uff[props].values[:, i],
    schnet_predictions_uff[i].flatten()) for i in range(8)},
                         name='b3lyp_schnet_uff')

mae_schnet_uff = mae_schnet_uff * 1000
mae_schnet_uff['spectral_overlap'] /= 1000
mae_schnet_uff.round(1)

gap                           45.1
homo                          33.1
lumo                          31.9
spectral_overlap             170.0
homo_extrapolated             64.8
lumo_extrapolated             63.0
gap_extrapolated              74.3
optical_lumo_extrapolated     60.2
Name: b3lyp_schnet_uff, dtype: float64

In [25]:
model_predictions = {
    '2d_multitarget': y_multitarget,
    '2d_singletarget': y_2d_st,
    'schnet': y_schnet,
    'schnet_uff': y_schnet_uff,
}

with open('model_predictions.p', 'wb') as f:
    pickle.dump(model_predictions, f)